In [ ]:
#top10kcity_meds

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import json
import math
import os
from fbprophet import Prophet
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error


In [5]:
path=os.getcwd()
path

'/content'

In [6]:
test=pd.read_csv(path +"/drive/My Drive/7879phd/test_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""],usecols=["city_medicine","date","id","year"])
train=pd.read_csv(path +"/drive/My Drive/7879phd/train_final.csv", sep=",", na_values=["?",",","#","NaN","unknown",""],usecols=["city_medicine","date","sales","year"])

In [7]:
train.head()

,year,date,sales,city_medicine
0,2015,2015-01-02,24.0,1_1
1,2015,2015-01-02,144.0,1_2
2,2015,2015-01-02,84.0,1_3
3,2015,2015-01-02,24.0,1_4
4,2015,2015-01-02,8.0,1_5


In [14]:
trn=train[train["year"]>=2017]# subset from 2017 only

In [24]:
sub_set=trn[["date","sales","city_medicine"]]

In [25]:
lst=list(trn["city_medicine"].value_counts().index)

In [26]:
sub_set.reset_index(inplace=True,drop=True)

In [28]:
sub_set.shape

(11719423, 3)

In [ ]:
def add_dates(lst):#adding missing dates
    final=pd.DataFrame()
    for med in lst:
        df=sub_set[sub_set["city_medicine"]==med]
        df["date"]=pd.to_datetime(df["date"])
        start_date1=pd.to_datetime("2017-01-02")
        end_date = pd.to_datetime("2018-06-30")
        dates=pd.DataFrame()
        dates["date"]=pd.date_range(start=start_date1,end=end_date,freq="d")
        dates=dates.merge(df,on=["date"],how="left")
        dates["city_medicine"]=med
        dates["sales"]=dates["sales"].fillna(0)
        #print(dates.head())
        final=pd.concat([final,dates])
    return final

In [4]:
new_trn_df=add_dates(lst)# let us save the train df

In [5]:
new_trn_df.to_csv("new_trn_df.csv",index=False)

In [6]:
new_trn_df=pd.read_csv(path +"/drive/My Drive/7879phd/new_trn_df.csv")

In [ ]:
new_trn_df.head()

,date,sales,city_medicine
0,2017-01-02,144.0,4_21
1,2017-01-03,160.0,4_21
2,2017-01-04,300.0,4_21
3,2017-01-05,152.0,4_21
4,2017-01-06,136.0,4_21


In [3]:
new_trn_df["date"]=pd.to_datetime(new_trn_df["date"])

In [1]:
def propht_model(lst):
    fin=pd.DataFrame()
    count=0
    for med in lst:
        df=new_trn_df[new_trn_df["city_medicine"]==med]
        df.columns = ['ds', 'y',"city_medicine"]
        df1 = df.loc[df.ds <= '2018-05-31']
        print(med)
        model = Prophet(interval_width = 0.95)
        model.fit(df1)
        count=count+1
        print(count)
        future= model.make_future_dataframe(periods=30)
        forecast = model.predict(future[-30:])
        forecast["city_medicine"]=med
        fin=pd.concat([fin,forecast[["ds","yhat","city_medicine"]]])
    return fin

Let us split the train data till may and make predictions on june and calculate rmse

In [15]:
jun_med_df=trn[(trn["date"]>="2018-06-01")&(trn["date"]<="2018-06-30")]["city_medicine"].value_counts().reset_index()

In [16]:
jun_med_df.shape

(30577, 2)

In [18]:
cmd_med_jun_df=jun_med_df[jun_med_df.index.isin(list(trn[trn["date"]<="2018-05-30"]["city_medicine"].values))].reset_index(drop=True)

In [19]:
cmd_med_jun_df.shape

(14264, 2)

In [43]:
ju=cmd_med_jun_df[cmd_med_jun_df.index.isin(list(test.city_medicine.value_counts().index))]

In [36]:
cmd_med_jun_df[cmd_med_jun_df.index.isin(list(test.city_medicine.value_counts().index))].shape

(11855, 2)

so there 11855 common medicines

In [39]:
df=pd.DataFrame(test["city_medicine"].value_counts(normalize=True))
df.rename(columns={"city_medicine":"percent"},inplace=True)
df["cumsum"]=df["percent"].cumsum()*100

In [51]:
df["valuecount"]=test["city_medicine"].value_counts().values

In [53]:
df[df["valuecount"]==31].shape

(10608, 3)

so all these medicines are avaibale in test data too

In [22]:
jun_lst=list(cmd_med_jun_df["index"])

In [23]:
len(jun_lst)

14264

In [20]:
np.unique(jun_med_df.index.isin(list(trn[trn["date"]<="2018-05-30"]["city_medicine"].values)),return_counts=True)

(array([False,  True]), array([16313, 14264]))

In [37]:
predj_1=propht_model(jun_lst[:5000])
predj_2=propht_model(jun_lst[5000:10000])
predj_3=propht_model(jun_lst[10000:14000])
predj_4=propht_model(jun_lst[14000:])

In [38]:
predj=pd.concat([predj_1,predj_2,predj_3,predj_4])

In [47]:
predj.to_csv("predj.csv",index=False)

In [48]:
def rmse(city_medlst):
    my={}
    for med in city_medlst:
            df=june_pred[june_pred["city_medicine"]==med].reset_index(drop=True)
            my[med]=np.sqrt(mean_squared_error(df["sales"],df["yhat"]))
    final=pd.DataFrame(my.items(),columns=["city_med","rmse"])
    return final

In [49]:
predj.rename(columns={"ds":"date"},inplace=True)
june_pred["date"]=pd.to_datetime(june_pred["date"])
trn=trn[["date","city_medicine","sales"]]
trn=trn[(trn["date"]>="2018-06-01")&(trn["date"]<="2018-06-30")].reset_index(drop=True)
june_pred=june_pred.merge(trn, on=["date","city_medicine"],how="right")
june_pred=june_pred[~june_pred["yhat"].isnull()]

In [2]:
rmse=rmse(rms_lst)

we could find rmse is low for al these medicines for prpht model when set a threshold of 50 and these are also part of test data so let us use this propht model on test data